### 0. Load environment variables
#### Case 1) If you use OpenAI's ChatGPT, then you need store the following variables in '.env' file:

- 'OPENAI_API_KEY' = XXXXXXXXXXXXXXX
- 'model_name' = XXXXXXXXXXXXXXX

##### Case 2) If you use AzureOpenAI's ChatGPT, then you need store the following variables in '.env' file:

- 'OPENAI_API_TYPE' = XXXXXXXXXXXXXXX
- 'OPENAI_API_VERSION' = XXXXXXXXXXXXXXX
- 'OPENAI_API_BASE' = XXXXXXXXXXXXXXX
- 'OPENAI_API_KEY' = XXXXXXXXXXXXXXX
- 'deployment_name' = XXXXXXXXXXXXXXX

In [1]:
import os
import sys  
from dotenv import load_dotenv
sys.path.extend(["..", '../..'])  
_ = load_dotenv('.env')


model_name = os.getenv('deployment_name') if os.getenv('OPENAI_API_TYPE') == 'azure' else os.getenv('model_name')
if os.getenv('OPENAI_API_TYPE') == 'azure':
    deployment_name = model_name

### 1. Create a scratchpad-compressing chain (using LCEL)

In [2]:
from langchain import hub
agent_scratchpad_compression_prompt = hub.pull("jet-taekyo-lee/agent-scratchpad-compression")

from langchain.chat_models import AzureChatOpenAI, ChatOpenAI
llm = AzureChatOpenAI(deployment_name=deployment_name, temperature=0) if os.getenv('OPENAI_API_TYPE') == 'azure' else ChatOpenAI(model_name=model_name, temperature=0)

agent_scratchpad_compression_chain = agent_scratchpad_compression_prompt | llm 

### 2. Fetch the prompt and create a chain (using LCEL)

In [3]:
from langchain import hub
prompt = hub.pull("jet-taekyo-lee/rag-style-react")

from utils.agent_components.configurations import Configurations
from utils.agent_tools.tools.web_search_tools import get_web_search_tools
from utils.agent_tools.tools.weather_tools import GetFromOpenWeatherMap
from utils.agent_tools.tools.time_tools import GetFromDatetimeModule
from utils.agent_tools.tools.python_repl_tools import GetLangChainPythonRepl
from langchain.tools.render import render_text_description_and_args
config = Configurations(provider='AzureChatOpenAI')
search_tool = get_web_search_tools(config)
weather_tool = GetFromOpenWeatherMap()
time_tool = GetFromDatetimeModule()
python_repl_tool = GetLangChainPythonRepl()
tools = [search_tool, weather_tool, time_tool, python_repl_tool]

from langchain.prompts import ChatPromptTemplate
prompt = prompt.partial(
        tools=render_text_description_and_args(tools),
        tool_names=", ".join([t.name for t in tools]),
)


from langchain.chat_models import AzureChatOpenAI, ChatOpenAI
llm = AzureChatOpenAI(deployment_name=deployment_name, temperature=0) if os.getenv('OPENAI_API_TYPE') == 'azure' else ChatOpenAI(model_name=model_name, temperature=0)

from langchain.schema.runnable import RunnablePassthrough
from utils.custom_output_parsers.agent_intermediatte_steps import format_intersteps_to_compressed_scratchpad
from langchain.agents.output_parsers import JSONAgentOutputParser

rag_style_react_agent = ( RunnablePassthrough.assign( compressed_agent_scratchpad = lambda x : format_intersteps_to_compressed_scratchpad(x["intermediate_steps"], agent_scratchpad_compression_chain) )
    | prompt | llm | JSONAgentOutputParser()
)

from langchain.agents import AgentExecutor
rag_style_react_agent_executor = AgentExecutor(agent=rag_style_react_agent, tools=tools, verbose=True)


### Get a response

In [4]:
rag_style_react_agent_executor.invoke({'input':'Find out the current time in New York City and then provide the weather forecast for the next 3 days in the same location.'})



> Entering new AgentExecutor chain...
Thought: The user wants to know the current time in New York City and the weather forecast for the next 3 days. I need to use the GetFromDatetimeModule tool to get the current time and the GetFromOpenWeatherMap tool to get the weather forecast. The current time is important to know because it will help me determine the start of the 3-day forecast period. 

Action:
```
{
"action": "GetFromDatetimeModule",
"action_input": {"IANA_timezone": "America/New_York"}
}
```timezone: America/New_York, current time(year-month-day-hour-minute): 2023-11-12-20-34Thought: The user has provided the current time in New York City, so I don't need to use the GetFromDatetimeModule tool. I can directly proceed to get the weather forecast for the next 3 days in New York City using the GetFromOpenWeatherMap tool. 

Action:
```
{
"action": "GetFromOpenWeatherMap",
"action_input": {"IANA_timezone": "America/New_York", "Days_from_now": 1}
}
```timezone: New York County, cur

{'input': 'Find out the current time in New York City and then provide the weather forecast for the next 3 days in the same location.',
 'output': 'The current time in New York City is 20:34 on November 12th, 2023. The weather forecast for the next 3 days is as follows: \n- November 13th, 2023: Partly cloudy with clear spells, a temperature of approximately 6.58 degrees, 42% humidity, and a wind speed of 4.57. \n- November 14th, 2023: Expected to be partly cloudy with clear spells, a temperature of approximately 6.58 degrees, humidity at 42%, and a wind speed of 4.57. \n- November 15th, 2023: Partly cloudy with clear spells, a temperature of approximately 7.92 degrees, 50% humidity, and a wind speed of 4.07.'}